✅ Parte 1: Preparação do Ambiente e dos Dados
O que está acontecendo:
Instalamos as dependências.

Carregamos o dataset JSON.

Preparamos os dados no formato de instrução-resposta (Instruction Tuning), onde:

Instrução: "Pergunta sobre título X: Y?"

Resposta: "Descrição do produto."



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Instalar dependências
!pip install -q transformers datasets peft bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.6 MB/s eta 0:00:00


In [4]:
import json

data = []
with open('/content/drive/MyDrive/Colab Notebooks/Datasets/trn.json') as f:
    for i, line in enumerate(f):
        data.append(json.loads(line))
        if i >= 9:  # pega só os 10 primeiros para inspeção
            break

print(data)

[{'uid': '0000031909', 'title': 'Girls Ballet Tutu Neon Pink', 'content': 'High quality 3 layer ballet tutu. 12 inches in length', 'target_ind': [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 111], 'target_rel': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}, {'uid': '0000032034', 'title': 'Adult Ballet Tutu Yellow', 'content': '', 'target_ind': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 16, 33, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71], 'target_rel': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}, {'uid': '0000913154', 'title': 'The Way Things Work: An Illustrated Encyclopedia 

In [5]:


# Importar bibliotecas
import json
import pandas as pd
from datasets import Dataset

data = []
with open('/content/drive/MyDrive/Colab Notebooks/Datasets/trn.json') as f:
    for i, line in enumerate(f):
        data.append(json.loads(line))
        if i >= 9:  # pega só os 10 primeiros para inspeção
            break

print(data)

# Converter em DataFrame
df = pd.DataFrame(data)

# Visualizar primeiras linhas
print(df.head())

# Preparar dados no formato de instrução
def prepare_instruction(row):
    return {
        'instruction': f"Pergunta sobre o título '{row['title']}': Descreva o produto.",
        'response': row['content']
    }

prepared_data = df.apply(prepare_instruction, axis=1).tolist()

# Converter para Hugging Face Dataset
dataset = Dataset.from_list(prepared_data)

# Ver exemplo
print(dataset[0])


[{'uid': '0000031909', 'title': 'Girls Ballet Tutu Neon Pink', 'content': 'High quality 3 layer ballet tutu. 12 inches in length', 'target_ind': [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 111], 'target_rel': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}, {'uid': '0000032034', 'title': 'Adult Ballet Tutu Yellow', 'content': '', 'target_ind': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 16, 33, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71], 'target_rel': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}, {'uid': '0000913154', 'title': 'The Way Things Work: An Illustrated Encyclopedia 

✅ Parte 2: Carregamento do Modelo Base e Avaliação Antes do Fine-Tuning
O que está acontecendo:
Carregamos um modelo base TinyLlama/TinyLlama-1.1B-Chat-v1.0.

Testamos como o modelo responde antes do fine-tuning.

Isso cria um baseline de comparação.

1️⃣ Instalar bibliotecas necessárias

In [6]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install sentencepiece


In [16]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

model_checkpoint = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
base_model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

generator = pipeline("text-generation", model=base_model, tokenizer=tokenizer)

prompt = "Question: What is Girls Ballet Tutu Neon?\nAnswer:"

result = generator(prompt, max_length=100, do_sample=True, temperature=0.7)
generated_text = result[0]['generated_text']
generated_only = generated_text[len(prompt):]

print("Generated Answer By TinyLlama Without Fine-Tuning Training:")
print(generated_only)


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Generated Answer By TinyLlama Without Fine-Tuning Training:
 Girls Ballet Tutu Neon is a lightweight, breathable, and stretchy tutu that is perfect for children's ballet performances and other dance events. It features bright neon colors and is designed to complement a child's outfit. The tutu is made of a blend of polyester and spandex, which makes it comfortable, flexible, and durable. The ne


✅ Parte 3: Fine-tuning com PEFT (LoRA)
O que está acontecendo:
Realizamos fine-tuning eficiente com LoRA, que evita treinar todos os parâmetros do modelo.

Criamos um Trainer e fazemos o ajuste.

Usamos poucos epochs, pois a ideia é apenas ilustrar.

ETAPAS ✈:

* Configura o LoRA	Define como o modelo vai ser adaptado (leve e eficiente)

* Aplica LoRA no modelo	Insere as camadas LoRA no modelo base

* Tokeniza os dados	Transforma dados (instruction + response) em tokens para o modelo

* Prepara batches	Usa o DataCollator para preparar os batches com padding e labels

* Define argumentos de treino	Como batch size, epochs, onde salvar etc.

* Cria Trainer	Junta tudo e gerencia o ciclo de treinamento
Executa treinamento	Ajusta os parâmetros LoRA com base nos dados


In [10]:
from peft import get_peft_model, LoraConfig, TaskType
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os
os.environ["WANDB_DISABLED"] = "true"

# Configuração LoRA
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1
)

# Aplicar PEFT no modelo
model = get_peft_model(model, peft_config)

# Tokenização
def tokenize_function(examples):
    combined = [
        instr + "\n" + resp
        for instr, resp in zip(examples['instruction'], examples['response'])
    ]
    return tokenizer(combined, truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Data Collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./finetuned_model",
    per_device_train_batch_size=1,
    num_train_epochs=1,  # Aumente para resultados reais
    save_steps=10,
    logging_steps=10
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Iniciar Fine-tuning
trainer.train()


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-10-3c017ffaa2d9>:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,3.485900


TrainOutput(global_step=10, training_loss=3.4858776092529298, metrics={'train_runtime': 61.5444, 'train_samples_per_second': 0.162, 'train_steps_per_second': 0.162, 'total_flos': 3100702519296.0, 'train_loss': 3.4858776092529298, 'epoch': 1.0})

✅ Esse resultado está bom?
Sim, considerando que o objetivo era testar e validar o pipeline de fine-tuning.

Para um fine-tuning real, normalmente usamos:

Mais épocas (num_train_epochs=3~10).

Ajuste de learning_rate.

Batch maior, se a infra permitir.

Avaliação em um conjunto de validação.

✅ Parte 4: Avaliação do Modelo Após o Fine-Tuning
O que está acontecendo:
Reutilizamos a pipeline.

Geramos resposta para o mesmo exemplo anterior.

Compararmos antes e depois do fine-tuning.



In [18]:
from transformers import pipeline

# NÃO precisa do merge_and_unload
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

input_prompt = "What is Girls Ballet Tutu Neon?"

# Inferência
output_after = pipe(input_prompt, max_new_tokens=100)

# Exibir
print("Resposta DEPOIS do fine-tuning:\n", output_after[0]['generated_text'])


Device set to use cpu


Resposta DEPOIS do fine-tuning:
 What is Girls Ballet Tutu Neon?
Girls Ballet Tutu Neon is a product that is designed to provide a fun and colorful experience for girls. It is a tutu that is designed to be worn by girls and is made from a neon fabric. The tutu is designed to be comfortable and easy to wear, and it is perfect for ballet classes or other dance performances. The neon fabric is a fun and eye-catching color that will make any girl feel like a ballerina.


🚨 Fluxo resumido:
Antes do fine-tuning: cria pipeline com modelo base e gera texto.

Depois do fine-tuning: aplica LoRA (merge_and_unload()), cria nova pipeline e gera texto.

Compara os resultados.



In [19]:
import torch
import math
from transformers import pipeline

def calculate_perplexity(model, tokenizer, input_text):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
        outputs = model(**inputs, labels=inputs['input_ids'])
        loss = outputs.loss
        perplexity = math.exp(loss.item())
    return perplexity

# --- Prompt para teste ---
input_prompt = "What is Girls Ballet Tutu Neon?"

# --- Inferência qualitativa (geração de texto) ---

# Pipeline do modelo base (antes do fine-tuning)
pipe_before = pipeline("text-generation", model=base_model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)
output_before = pipe_before(input_prompt, max_new_tokens=100)
print("Resposta ANTES do fine-tuning:\n", output_before[0]['generated_text'])

# Pipeline do modelo fine-tuned (sem merge_and_unload)
pipe_after = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)
output_after = pipe_after(input_prompt, max_new_tokens=100)
print("\nResposta DEPOIS do fine-tuning:\n", output_after[0]['generated_text'])

# --- Comparação Quantitativa: Perplexity ---

# Perplexity antes do fine-tuning
ppl_before = calculate_perplexity(base_model, tokenizer, input_prompt)
print(f"\nPerplexity ANTES do fine-tuning: {ppl_before:.4f}")

# Perplexity depois do fine-tuning
ppl_after = calculate_perplexity(model, tokenizer, input_prompt)
print(f"Perplexity DEPOIS do fine-tuning: {ppl_after:.4f}")

# --- Avaliação ---
improvement = ppl_before - ppl_after
print(f"\nDiferença de Perplexity (antes - depois): {improvement:.4f}")


Device set to use cpu
Device set to use cpu


Resposta ANTES do fine-tuning:
 What is Girls Ballet Tutu Neon?
Girls Ballet Tutu Neon is a product that is designed to provide a fun and exciting way for girls to dance. The tutu is made from a soft and stretchy material that is perfect for dancing. The tutu is designed to be comfortable and easy to move in. The tutu is also designed to be lightweight, so that girls can move around easily and not feel weighed down.
The tutu is available in a variety of colors, including

Resposta DEPOIS do fine-tuning:
 What is Girls Ballet Tutu Neon?
Girls Ballet Tutu Neon is a product that is designed to provide a fun and colorful experience for girls. It is a tutu that is designed to be worn by girls and is made from a neon fabric. The tutu is designed to be comfortable and easy to wear, and it is perfect for ballet classes or other dance performances. The neon fabric is a fun and eye-catching color that will make any girl feel like a ballerina.

Perplexity ANTES do fine-tuning: 287.6886
Perplexity

Análise rápida do resultado:
Geração antes do fine-tuning: Texto coerente, mas mais genérico e repetitivo.

Geração depois do fine-tuning: Texto mais específico, com termos mais alinhados ao prompt ("colorful experience", "neon fabric"), indicando que o modelo aprendeu nuances do seu dataset.

Perplexity: Caiu de 287.7 para 284.2, uma melhora modesta, que indica o modelo está um pouco mais confiante na previsão dos tokens após o fine-tuning.